In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import pandas as pd
from functools import reduce
from asymmetree.treeevolve.SpeciesTree import species_tree_n
from asymmetree.tools.PhyloTreeTools import (to_newick,)
from tralda.datastructures.Tree import Tree

class TransferManager:
    def __init__(self):
        pass

    def add_transfer_to_tree(self, G, pw, w, pay, ay, c, attributes):

        """
            Ajoute un transfert au graphe G.

            Cette fonction effectue les étapes nécessaires pour ajouter un transfert au graphe G. Elle supprime d'abord les arêtes
            entre les nœuds pw, w, pay et ay, puis ajoute de nouveaux nœuds et arêtes pour représenter le transfert. Elle attribue
            également des timestamps aux nouveaux nœuds et calcule les étiquettes de caractères des nouveaux nœuds en utilisant
            les étiquettes de caractères des nœuds existants.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX contenant les nœuds et les arêtes.
                pw (int): L'identifiant du nœud parent direct de w.
                w (int): L'identifiant du nœud w.
                pay (int): L'identifiant du nœud parent direct de ay.
                ay (int): L'identifiant du nœud ay.
                c (int): Le caractère à ajouter lors du transfert.

            Returns:
                networkx.Graph: Le graphe G mis à jour avec le transfert ajouté.
        """

        # Ensemble pour stocker les timestamps
        timestamps = set()
        # Parcourir les nœuds du graphe G et ajouter les timestamps à l'ensemble timestamps
        for node in G.nodes():
            timestamps.add(G.nodes[node]['timestamp'])

        # Suppression des arêtes entre pw, w, pay et ay
        G.remove_edge(pw, w)
        G.remove_edge(pay, ay)

        # Ajout des nouveaux nœuds pour représenter le transfert
        new_node_1 = max(G.nodes()) + 1
        new_node_2 = max(G.nodes()) + 2

        G.add_edge(pw, new_node_1, type='support', color='black')
        G.add_edge(new_node_1, w, type='support', color='black')
        G.add_edge(pay, new_node_2, type='support', color='black')
        G.add_edge(new_node_2, ay, type='support', color='black')


        # Ajout d'une arête entre ces deux nouveaux nœuds
        G.add_edge(new_node_1, new_node_2)
        G.nodes[new_node_1]['type']='network'
        G.nodes[new_node_2]['type']='network'
        # Ajout de l'attribut "type" à la nouvelle arête
        G[new_node_1][new_node_2]['type'] = 'transfer'
        # Attribution de l'attribut "color" à la nouvelle arête
        G[new_node_1][new_node_2]['color'] = 'red'
        # Attribution de l'attribut "caractere" à la nouvelle arête
        G[new_node_1][new_node_2]['character'] = c+1



        # Calcul des timestamps des nouveaux nœuds
          # timestamp des nouveaux noeuds= Moyenne de l'intervalle [max(timestamp de node_1 , timestamp de node_3) , min(timestamp de node_2 , timestamp de node_4)]
        timestamp_manager = TimestampManager()
        mean_timestamp = timestamp_manager.calculate_mean_timestamp(G, pw, w, pay, ay)

        if mean_timestamp in timestamps:  # Si le "mean_timestamp" existe déjà dans G
          epsilon = 1e-6  # Choisir la valeur d'épsilon appropriée
          mean_timestamp += epsilon
        G.nodes[new_node_1]['timestamp']=mean_timestamp
        G.nodes[new_node_2]['timestamp']=mean_timestamp



        # Ajout du numéro des nouveaux nœuds comme attribut
        G.nodes[new_node_1]["number"] = max(nx.get_node_attributes(G, "number").values()) + 1
        G.nodes[new_node_2]["number"] = max(nx.get_node_attributes(G, "number").values()) + 1

        # Étiquetage des nouveaux nœuds:

        #l( ˆ w) = (l(w) ∩ l(w′)
        Characters_new_node_1 = [a & b for a, b in zip(attributes[G.nodes[w]["number"]]['Characters'], attributes[G.nodes[pw]["number"]]['Characters'])]
        # (l(ˆ ai+1) = l(a′ i+1) ∩ l(ai+1))
        Characters_new_node_2 = [a & b for a, b in zip(attributes[G.nodes[ay]["number"]]['Characters'], attributes[G.nodes[pay]["number"]]['Characters'])]

        # Ajout des nouvelles étiquettes de caractères aux nœuds nouvellement créés
        attributes[G.nodes[new_node_1]["number"]] = {'Characters': Characters_new_node_1, 'spece': None}
        attributes[G.nodes[new_node_2]["number"]] = {'Characters': Characters_new_node_2, 'spece': None}

        # Ajout du caractère c aux étiquettes de caractères des nouveaux nœuds
        # l( ˆ w) = (l(w) ∩ l(w′)) ∪ {c}  et  (l(ˆ ai+1) = l(a′ i+1) ∩ l(ai+1)) ∪ {c}
        c_list = [1 if i == c else 0 for i in range(len(attributes[G.nodes[pw]["number"]]['Characters']))]
        print(c_list)
        attributes[G.nodes[new_node_1]["number"]]['Characters'] = [a | b for a, b in zip(attributes[G.nodes[new_node_1]["number"]]['Characters'], c_list)]
        attributes[G.nodes[new_node_2]["number"]]['Characters'] = [a | b for a, b in zip(attributes[G.nodes[new_node_2]["number"]]['Characters'], c_list)]


        return G



    def extract_bfs_tree_without_transfer(self, G, ai):

        """
            Extrait l'arbre BFS du graphe G à partir du nœud 'ai', en supprimant toutes les arêtes de transfert du BFS.

            Cette fonction calcule l'arbre BFS du graphe G à partir du nœud 'ai', puis supprime toutes les arêtes de transfert
            du BFS obtenu. Elle renvoie ensuite les nœuds de cet arbre BFS sans les nœuds connectés à des arêtes de transfert.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX contenant les nœuds et les arêtes.
                ai (int): L'identifiant du nœud à partir duquel l'arbre BFS doit être extrait.

            Returns:
                list: Une liste de nœuds représentant l'arbre BFS sans les nœuds connectés à des arêtes de transfert.
        """

        # Obtenir l'arbre BFS à partir du nœud ai
        bfs_tree = nx.bfs_tree(G, ai)

        # Supprimer les arêtes de type 'transfer' de bfs_tree
        edges_to_remove = [(u, v) for u, v in bfs_tree.edges() if G.edges[u, v]['type'] == 'transfer']
        bfs_tree.remove_edges_from(edges_to_remove)

        # Obtenir l'arbre BFS à partir du nœud ai => la partie qui reste connectée à la racine ai
        bfs_tree_ai = nx.bfs_tree(bfs_tree, ai)

        # Récupérer les nœuds dans l'ordre du parcours en largeur (BFS)
        bfs_nodes = list(bfs_tree_ai.nodes())

        return bfs_nodes




    def add_transfers_to_tree_PTN(self, G):
        """
            Effectue une série d'ajouts aléatoires de branches du type transfert sur le graphe G.

            Cette fonction effectue un nombre aléatoire d'itérations pour ajouter aléatoirement des branches du type transfert
            au graphe G en supprimant certaines arêtes existantes et en ajoutant de nouveaux nœuds et arêtes.
            Les nouvelles branches sont choisies de manière à ce qu'elles ne créent pas de relation de descendance
            entre les nœuds nouvellement ajoutés et en respectant les conditions concernant les timestamps des nouveaux nœuds :
            Premièrement, garantit que les nouveaux nœuds ont des périodes temporelles cohérentes avec ceux dont ils sont issus.
            Deuxièmement, l'absence de redondances temporelles.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX.

            Returns:
                networkx.DiGraph: Le graphe G avec les ajouts aléatoires de branches.
        """

        # Répéter un nombre aléatoire de fois entre 1 et 4
        num_iterations = random.randint(1, 4)

        # Ensemble pour stocker les timestamps : chaque élément ne peut apparaître qu'une seule fois.Si un élément déjà présent dans l'ensemble est ajouté à nouveau, il ne sera pas inclus une seconde fois.
        timestamps = set(node_data['timestamp'] for _, node_data in G.nodes(data=True))


        for _ in range(num_iterations):
            # Variables pour garder une trace des nœuds ajoutés
            new_node_1 = None
            new_node_2 = None

            # Variable pour indiquer si les nœuds ajoutés sont valides
            found = False

            # Répéter jusqu'à ce que les nœuds ajoutés ne soient pas en relation de descendance l'un de l'autre et respectent les conditions relatives aux timestamps.
            while not found:
                # Choix aléatoire de deux arêtes existantes
                edges = [edge for edge in G.edges() if G[edge[0]][edge[1]]['type'] == 'support']
                random_edge_1, random_edge_2 = random.sample(edges, 2)

                node_1, node_2 = random_edge_1
                node_3, node_4 = random_edge_2

                # Variables pour garder une trace des anciens nœuds
                number_node_1= G.nodes[node_1]['number']
                number_node_2= G.nodes[node_2]['number']
                number_node_3= G.nodes[node_3]['number']
                number_node_4= G.nodes[node_4]['number']

                timeStamp_node_1= G.nodes[node_1]['timestamp']
                timeStamp_node_2= G.nodes[node_2]['timestamp']
                timeStamp_node_3= G.nodes[node_3]['timestamp']
                timeStamp_node_4= G.nodes[node_4]['timestamp']

                # Suppression des arêtes choisies
                G.remove_edge(node_1, node_2)
                G.remove_edge(node_3, node_4)

                # Ajout des nouveaux nœuds et des arêtes
                new_node_1 = max(G.nodes()) + 1
                new_node_2 = max(G.nodes()) + 2

                G.add_edge(node_1, new_node_1, type='support', color='black')
                G.add_edge(new_node_1, node_2, type='support', color='black')
                G.add_edge(node_3, new_node_2, type='support', color='black')
                G.add_edge(new_node_2, node_4, type='support', color='black')

                G.nodes[new_node_1]['type']='network'
                G.nodes[new_node_2]['type']='network'

                # Calcul du mean_timestamp
                timestamp_manager = TimestampManager()
                mean_timestamp = timestamp_manager.calculate_mean_timestamp(G, node_1, node_2, node_3, node_4)

                # Vérification si aucun nœud n'est un descendant de l'autre
                descendants_1 = nx.descendants(G, new_node_1)
                descendants_2 = nx.descendants(G, new_node_2)


                if (new_node_1 not in descendants_2 and new_node_2 not in descendants_1)and \
                      timestamp_manager.check_intersection(G.nodes[node_1]['timestamp'], G.nodes[node_2]['timestamp'],
                                          G.nodes[node_3]['timestamp'], G.nodes[node_4]['timestamp'])and \
                      mean_timestamp not in timestamps:
                    # Ajout d'une arête entre les deux nouveaux nœuds
                    G.add_edge(new_node_1, new_node_2, type='transfer', color='red')
                    # Ajout de l'attribut "type" à la nouvelle arête
                    G[new_node_1][new_node_2]['type'] = 'transfer'
                    G[new_node_1][new_node_2]['color'] = 'red'

                    # Mise à jour des timestamps des nouveaux nœuds
                    G.nodes[new_node_1]['timestamp']=mean_timestamp
                    G.nodes[new_node_2]['timestamp']=mean_timestamp

                    # Ajout du mean_timestamp à l'ensemble des timestamps
                    timestamps.add(mean_timestamp)
                    # Ajout du numéro des nouveaux nœuds comme attribut
                    max_number = max(nx.get_node_attributes(G, "number").values())
                    G.nodes[new_node_1]["number"] = max_number + 1
                    G.nodes[new_node_2]["number"] = max_number + 2

                    # Si les conditions pour ajouter les nœuds sont vérifiées
                    found = True
                else:
                    # Suppression des nouveaux nœuds ajoutés et restauration des anciens nœuds et arêtes à leur emplacement initial.
                    G.remove_node(new_node_1)
                    G.remove_node(new_node_2)
                    G.add_edge(node_1, node_2, type='support', color='black')
                    G.add_edge(node_3, node_4, type='support', color='black')
                    G.nodes[node_1]['number']=number_node_1
                    G.nodes[node_2]['number']=number_node_2
                    G.nodes[node_3]['number']=number_node_3
                    G.nodes[node_4]['number']=number_node_4

                    G.nodes[node_1]['timestamp']=timeStamp_node_1
                    G.nodes[node_2]['timestamp']=timeStamp_node_2
                    G.nodes[node_3]['timestamp']=timeStamp_node_3
                    G.nodes[node_4]['timestamp']=timeStamp_node_4


        # Affichage
        plt.title("Arbre depuis le format Newick avec modification des arêtes")
        tree_builder = TreeBuilder()
        tree_builder.drawGraph(G)

        return G



    def add_transfers_to_tree_Non_PTN(self, G, x, y):

        """
            Ajoute des transferts aléatoires entre les branches d'un graphe d'arbre G
            tout en respectant certaines conditions pour obtenir un graphe non PTN.

            Args:
                G (networkx.Graph): Le graphe d'arbre NetworkX.
                x, y: Les nœuds pour lesquels nous voulons ajouter des transferts.

            Returns:
                networkx.Graph: Le graphe d'arbre avec les transferts ajoutés.
        """
        # Répéter un nombre aléatoire de fois
        num_iterations = random.randint(1, 4)

        # Ensemble pour stocker les timestamps : chaque élément ne peut apparaître qu'une seule fois.Si un élément déjà présent dans l'ensemble est ajouté à nouveau, il ne sera pas inclus une seconde fois.
        timestamps = set(node_data['timestamp'] for _, node_data in G.nodes(data=True))



        for _ in range(num_iterations):
            # Variables pour garder une trace des nœuds ajoutés
            new_node_1 = None
            new_node_2 = None

            # Variable pour indiquer si les nœuds ajoutés sont valides
            found = False
            # Répéter jusqu'à ce que les nœuds ajoutés ne soient pas en relation de descendance l'un de l'autre et respectent les conditions relatives aux timestamps.
            while not found:

                # Choix aléatoire de deux arêtes existantes avec type='support'
                edges = [edge for edge in G.edges() if G[edge[0]][edge[1]]['type'] == 'support']
                random_edge_1 = random.choice(edges)
                edges.remove(random_edge_1)  # Retirer la première arête sélectionnée de la liste
                random_edge_2 = random.choice(edges)

                node_1, node_2 = random_edge_1
                node_3, node_4 = random_edge_2

                # Variables pour garder une trace des anciens nœuds
                number_node_1= G.nodes[node_1]['number']
                number_node_2= G.nodes[node_2]['number']
                number_node_3= G.nodes[node_3]['number']
                number_node_4= G.nodes[node_4]['number']

                timeStamp_node_1= G.nodes[node_1]['timestamp']
                timeStamp_node_2= G.nodes[node_2]['timestamp']
                timeStamp_node_3= G.nodes[node_3]['timestamp']
                timeStamp_node_4= G.nodes[node_4]['timestamp']

                # Suppression des arêtes choisies
                G.remove_edge(node_1, node_2)
                G.remove_edge(node_3, node_4)

                # Ajout des nouveaux nœuds et des arêtes
                new_node_1 = max(G.nodes()) + 1
                new_node_2 = max(G.nodes()) + 2

                G.add_edge(node_1, new_node_1, type='support', color='black')
                G.add_edge(new_node_1, node_2, type='support', color='black')
                G.add_edge(node_3, new_node_2, type='support', color='black')
                G.add_edge(new_node_2, node_4, type='support', color='black')

                G.nodes[new_node_1]['type']='network'
                G.nodes[new_node_2]['type']='network'

                # Vérification si aucun nœud n'est un descendant de l'autre
                descendants_1 = nx.descendants(G, new_node_1)
                descendants_2 = nx.descendants(G, new_node_2)

                # Calcul du mean_timestamp
                timestamp_manager = TimestampManager()
                mean_timestamp = timestamp_manager.calculate_mean_timestamp(G, node_1, node_2, node_3, node_4)


                descendants_x = set()
                descendants_y = set()
                px = next(G.predecessors(x)) # Récupère le premier prédécesseur direct (parent) du nœud x. Note : puisque nous travaillons avec un arbre, il n'y a qu'un seul prédécesseur direct.)
                py = next(G.predecessors(y)) # Récupère le premier prédécesseur direct (parent) du nœud y.
                descendants_x = set(nx.descendants(G, x)) | {x, px}   #RQ: Ajoute également les nœuds x et px à cet ensemble : " | {x, px}"
                descendants_y = set(nx.descendants(G, y)) | {y, py}


                # Vérification des conditions pour ajouter les nœuds:
                  # Les nœuds ne sont pas en relation de descendance l'un de l'autre et respectent les conditions relatives aux timestamps
                  # Assurer que new_node_1 ne soit pas un descendant de px (parent de x) et que new_node_2 ne soit pas un descendant de py (parent de y), et vice versa
                if (new_node_1 not in descendants_2 and new_node_2 not in descendants_1) and \
                  timestamp_manager.check_intersection(G.nodes[node_1]['timestamp'], G.nodes[node_2]['timestamp'],
                                      G.nodes[node_3]['timestamp'], G.nodes[node_4]['timestamp']) and \
                  mean_timestamp not in timestamps and \
                  (new_node_1 not in descendants_x and new_node_2 not in descendants_y) and \
                  (new_node_2 not in descendants_x and new_node_1 not in descendants_y):

                    # Ajout de la nouvelle arête entre les deux nouveaux nœuds
                    G.add_edge(new_node_1, new_node_2, type='transfer', color='red')


                    # timestamp des nouveaux noeuds= Moyenne de l'intervalle [max(timestamp de node_1 , timestamp de node_3) , min(timestamp de node_2 , timestamp de node_4)]
                    # Mise à jour des timestamps des nouveaux nœuds
                    G.nodes[new_node_1]['timestamp']=mean_timestamp
                    G.nodes[new_node_2]['timestamp']=mean_timestamp

                    # Ajout du mean_timestamp à l'ensemble des timestamps
                    timestamps.add(mean_timestamp)

                    # Ajout des attributs "number" pour les nouveaux nœuds
                    max_number = max(nx.get_node_attributes(G, "number").values()) + 1
                    G.nodes[new_node_1]["number"] = max_number
                    G.nodes[new_node_2]["number"] = max_number + 1

                    # Si les conditions pour ajouter les nœuds sont vérifiée
                    found = True
                else:
                    # Suppression des nouveaux nœuds ajoutés et restauration des anciens nœuds et arêtes à leur emplacement initial.
                    G.remove_node(new_node_1)
                    G.remove_node(new_node_2)
                    G.add_edge(node_1, node_2, type='support', color='black')
                    G.add_edge(node_3, node_4, type='support', color='black')
                    G.nodes[node_1]['number']=number_node_1
                    G.nodes[node_2]['number']=number_node_2
                    G.nodes[node_3]['number']=number_node_3
                    G.nodes[node_4]['number']=number_node_4

                    G.nodes[node_1]['timestamp']=timeStamp_node_1
                    G.nodes[node_2]['timestamp']=timeStamp_node_2
                    G.nodes[node_3]['timestamp']=timeStamp_node_3
                    G.nodes[node_4]['timestamp']=timeStamp_node_4


        return G



    def determine_transfers(self, G):
        """
            Détermine chaque transfert dans le graphe G en considérant les arêtes de support.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX contenant les nœuds et les arêtes.

            Returns:
                list: Une liste de tuples, chaque tuple contient deux ensembles. Le premier ensemble représente les feuilles
                      atteignables par le nœud de départ du transfert (en utilisant les arêtes de support), et le deuxième ensemble
                      représente les feuilles atteignables par le nœud d'arrivée du transfert (en utilisant les arêtes de support).
        """
        transfers = []

        for source_node, target_node, data in G.edges(data=True):
            if data.get('type') == 'transfer':
                source_leaves = set()
                target_leaves = set()

                # Parcours en profondeur pour trouver les feuilles atteignables par le nœud de départ du transfert (en utilisant les arêtes de support)
                for leaf in nx.descendants(G, source_node):
                    if G.out_degree(leaf) == 0 and nx.has_path(G, source_node, leaf):
                        for path in nx.edge_disjoint_paths(G, source_node, leaf):
                            edges = list(zip(path, path[1:]))
                            if all(G.edges[u, v]['type'] == 'support' for u, v in edges):
                                source_leaves.add(leaf)

                # Parcours en profondeur pour trouver les feuilles atteignables par le nœud d'arrivée du transfert (en utilisant les arêtes de support)
                for leaf in nx.descendants(G, target_node):
                    if G.out_degree(leaf) == 0 and nx.has_path(G, target_node, leaf):
                        for path in nx.edge_disjoint_paths(G, target_node, leaf):
                            edges = list(zip(path, path[1:]))
                            if all(G.edges[u, v]['type'] == 'support' for u, v in edges):
                                target_leaves.add(leaf)

                transfers.append(((source_node, target_node), source_leaves, target_leaves))

        return transfers



    def is_PTN(self, G, attributes_eliminate, nbrCaractere):
        """
          Vérifie si le graphe reste un réseau phylogénétique (PTN) après la suppression d'une arêtes.

          Args:
              G (networkx.DiGraph): Le graphe NetworkX contenant les nœuds et les arêtes.
              attributes_eliminate (dict): Un dictionnaire contenant les attributs des nœuds dans le graphe G après l'élimination des arêtes.
              nbrCaractere (int): Le nombre de caractères utilisés pour l'étiquetage des nœuds.

          Returns:
              bool: True si le graphe reste un PTN, False sinon.
        """
        # Appelez la fonction findLabeling pour tenter de résoudre le problème de reconnaissance d'évolution de PTN
        # Liste des caractères à vérifier
        C = [f'c{i+1}' for i in range(nbrCaractere)]
        tree_networks = TreeBasedNetworks()
        attributes_eliminate = tree_networks.findLabeling(G, attributes_eliminate, C)
        print("attributes_eliminate : ",attributes_eliminate)
        # Si le résultat "attributes_eliminate" est None, cela signifie que l'étiquetage n'est pas possible, donc le graphe n'est pas un PTN
        return attributes_eliminate is not None

    # Fonction pour éliminer progressivement les arêtes "transfer" jusqu'à ce que le graphe ne soit plus un PTN
    def eliminate_transfer_edges(self, G, attributes, nbrCaractere):
        """
            Cette fonction permet d'eliminer les arêtes qui sont potentiellement dispensables dans le réseau phylogénétique,
            de sorte que si ces arêtes sont éliminées, le graphe résultant reste un Phylogenetic Network (PTN).

            Args:
                G (networkx.DiGraph): Le graphe NetworkX contenant les nœuds et les arêtes.
                attributes (dict): Un dictionnaire contenant les attributs des nœuds du réseau G.
                nbrCaractere (int): Le nombre de caractères utilisés pour l'étiquetage des nœuds.

            Returns:
                G (networkx.DiGraph): Le graphe modifié après les éliminations d'arêtes.
                dict: Les attributs mis à jour des nœuds après les éliminations d'arêtes.
        """

        def compare_transfer_edges(edge):
          # Comparaison des arêtes "transfer" en utilisant le timestamp de leurs nœuds
          u, v = edge
          return G.nodes[u]['timestamp'] - G.nodes[v]['timestamp']

        eliminated_edges_count = 0
        is_PTN_graph = True  # Initialiser la variable booléenne à True
        attributes_eliminate= attributes
        while is_PTN_graph:  # Boucle while utilisant la variable booléenne : permet de continuer tant que le graphe est considéré comme un PTN (c'est-à-dire que is_PTN_graph est True)

            transfer_manager = TransferManager()
            transfer_edges = [(u, v) for u, v, data in G.edges(data=True) if data.get('type') == 'transfer']
            sorted_transfer_edges = sorted(transfer_edges, key= compare_transfer_edges)


            if not sorted_transfer_edges:
                # S'il n'y a plus d'arêtes "transfer", on arrête
                print("il n'y a plus d'arêtes 'transfer'")
                break

            for edge_to_remove in sorted_transfer_edges:
                # Supprimez une arête de type "transfer" du graphe
                u, w = edge_to_remove

                # Trouver les prédécesseurs et les successeurs de u et w qui ont le type 'support'
                predecessors_u = [pred for pred in G.predecessors(u) if G.edges[pred, u]['type'] == 'support']
                successors_u = [succ for succ in G.successors(u) if G.edges[u, succ]['type'] == 'support']
                predecessors_w = [pred for pred in G.predecessors(w) if G.edges[pred, w]['type'] == 'support']
                successors_w = [succ for succ in G.successors(w) if G.edges[w, succ]['type'] == 'support']

                print("u: ",G.nodes[u]["number"],"w: ",G.nodes[w]["number"])

                # Assumer qu'il y a un seul prédécesseur et un seul successeur pour u et w
                parent_u, child_u = predecessors_u[0], successors_u[0]
                parent_w, child_w = predecessors_w[0], successors_w[0]

                # Ajoutez une nouvelle arête entre le parent du nœud u et l'enfant du nœud w
                G.add_edge(parent_u, child_u, type='support', color='black')

                # Ajoutez une nouvelle arête entre le parent du nœud w et l'enfant du nœud u
                G.add_edge(parent_w, child_w, type='support', color='black')

                G.remove_edge(*edge_to_remove)


                eliminated_edges_count += 1


                timeStamp_u = G.nodes[u]['timestamp']
                timeStamp_w = G.nodes[w]['timestamp']
                number_u=G.nodes[u]['number']
                number_w=G.nodes[w]['number']
                G.remove_node(u)
                G.remove_node(w)


                labeler = LeafLabeler()
                attributes_av= attributes_eliminate
                # Mettre à jour la liste attribute
                attributes_eliminate = labeler.set_leaf_attributes(G, nbrCaractere) # sans les deux noeuds de TR, contient etiquetage des feuilles seulements


                if not transfer_manager.is_PTN(G, attributes_eliminate, nbrCaractere):  # Non PTN à partir de de l"elimination de la derniere arete

                  #remettre tous dans le cas Non_PTN
                  eliminated_edges_count -= 1
                  print("eliminated_edges_count: ", eliminated_edges_count)
                  is_PTN_graph = False  # Mettre la variable à False pour sortir de la boucle while

                  # eliminer les nouvelles aretes "supprot"
                  G.remove_edge(parent_u, child_u)
                  G.remove_edge(parent_w, child_w)
                  # remettre les arete avec u et w
                  G.add_edge(parent_u, u, type='transfer', color='red')
                  G.add_edge(u, child_u, type='transfer', color='red')
                  G.add_edge(parent_w, w, type='transfer', color='red')
                  G.add_edge(w, child_w, type='transfer', color='red')
                  G.nodes[u]['number']=number_u
                  G.nodes[w]['number']=number_w
                  G.nodes[u]['timestamp']=timeStamp_u
                  G.nodes[w]['timestamp']=timeStamp_w
                  # on revient à la derniere etiquetage possible
                  attributes_eliminate= attributes_av

                  break  # Sortir de la boucle for si le graphe n'est plus un PTN

                print(attributes_eliminate)

        return G, attributes_eliminate




    def eliminate_transfer_edges_random(self, G, attributes, nbrCaractere):
        """
              Cette fonction permet d'eliminer les arêtes qui sont potentiellement dispensables dans le réseau phylogénétique,
              de sorte que si ces arêtes sont éliminées, le graphe résultant reste un Phylogenetic Network (PTN).

              1- Trie les arêtes de transfert en fonction des timestamps de leurs nœuds.
              2- Parcours les arêtes de transfert en mélangeant à chaque fois, d'une façon alétoire, la liste d'arêtes et les élimine progressivement, en vérifiant à chaque étape si le graphe résultant reste un PTN.
              3- Si l'élimination d'une arête de transfert conduit à un graphe qui n'est pas un PTN, la dernière élimination est annulée pour rétablir l'état précédent.
              4- La fonction retourne le graphe modifié ainsi que les attributs des nœuds mis à jour après les éliminations d'arêtes.

              Args:
                  G (networkx.DiGraph): Le graphe NetworkX contenant les nœuds et les arêtes.
                  attributes (dict): Un dictionnaire contenant les attributs des nœuds du réseau G.
                  nbrCaractere (int): Le nombre de caractères utilisés pour l'étiquetage des nœuds.

              Returns:
                  G (networkx.DiGraph): Le graphe modifié après les éliminations d'arêtes.
                  dict: Les attributs mis à jour des nœuds après les éliminations d'arêtes.
          """
        eliminated_edges_count = 0
        is_PTN_graph = True  # Initialiser la variable booléenne à True
        attributes_eliminate= attributes
        while is_PTN_graph:  # Boucle while utilisant la variable booléenne : permet de continuer tant que le graphe est considéré comme un PTN (c'est-à-dire que is_PTN_graph est True)


            transfer_edges = [(u, v) for u, v, data in G.edges(data=True) if data.get('type') == 'transfer']

            if not transfer_edges:
                # S'il n'y a plus d'arêtes "transfer", on arrête
                print("il n'y a plus d'arêtes 'transfer'")
                break

            # Mélanger aléatoirement la liste des edges de type 'transfer' pour avoir un ordre aléatoire
            random.shuffle(transfer_edges)

            for edge_to_remove in transfer_edges:
                # Supprimez une arête de type "transfer" du graphe
                u, w = edge_to_remove

                # Trouver les prédécesseurs et les successeurs de u et w qui ont le type 'support'
                predecessors_u = [pred for pred in G.predecessors(u) if G.edges[pred, u]['type'] == 'support']
                successors_u = [succ for succ in G.successors(u) if G.edges[u, succ]['type'] == 'support']
                predecessors_w = [pred for pred in G.predecessors(w) if G.edges[pred, w]['type'] == 'support']
                successors_w = [succ for succ in G.successors(w) if G.edges[w, succ]['type'] == 'support']

                print("u: ",G.nodes[u]["number"],"w: ",G.nodes[w]["number"])

                # Assumer qu'il y a un seul prédécesseur et un seul successeur pour u et w
                parent_u, child_u = predecessors_u[0], successors_u[0]
                parent_w, child_w = predecessors_w[0], successors_w[0]

                print("parent_u: ",G.nodes[parent_u]["number"],"child_u: ",G.nodes[child_u]["number"])
                print("parent_w: ",G.nodes[parent_w]["number"],"child_w: ",G.nodes[child_w]["number"])

                # Ajoutez une nouvelle arête entre le parent du nœud u et l'enfant du nœud w
                G.add_edge(parent_u, child_u, type='support', color='black')

                # Ajoutez une nouvelle arête entre le parent du nœud w et l'enfant du nœud u
                G.add_edge(parent_w, child_w, type='support', color='black')

                G.remove_edge(*edge_to_remove)

                eliminated_edges_count += 1

                timeStamp_u = G.nodes[u]['timestamp']
                timeStamp_w = G.nodes[w]['timestamp']
                number_u=G.nodes[u]['number']
                number_w=G.nodes[w]['number']
                G.remove_node(u)
                G.remove_node(w)


                labeler = LeafLabeler()
                attributes_av= attributes_eliminate
                # Mettre à jour la liste attribute
                attributes_eliminate = labeler.set_leaf_attributes(G, nbrCaractere)

                transfer_manager = TransferManager()
                if not transfer_manager.is_PTN(G, attributes_eliminate, nbrCaractere):

                  #remettre tous dans le cas Non_PTN
                  eliminated_edges_count -= 1
                  print("eliminated_edges_count: ", eliminated_edges_count)
                  is_PTN_graph = False  # Mettre la variable à False pour sortir de la boucle while

                  # eliminer les nouvelles aretes "supprot"
                  G.remove_edge(parent_u, child_u)
                  G.remove_edge(parent_w, child_w)
                  # remettre les arete avec u et w
                  G.add_edge(parent_u, u, type='transfer', color='red')
                  G.add_edge(u, child_u, type='transfer', color='red')
                  G.add_edge(parent_w, w, type='transfer', color='red')
                  G.add_edge(w, child_w, type='transfer', color='red')
                  G.nodes[u]['number']=number_u
                  G.nodes[w]['number']=number_w
                  G.nodes[u]['timestamp']=timeStamp_u
                  G.nodes[w]['timestamp']=timeStamp_w
                  # on revient à la derniere etiquetage possible
                  attributes_eliminate= attributes_av

                  break  # Sortir de la boucle for si le graphe n'est plus un PTN

                print(attributes_eliminate)

        return G, attributes_eliminate, eliminated_edges_count


class TimestampManager:
    def __init__(self):
        pass

    def add_timestamp(self, G, root_id):
        """
            Ajoute des timestamps aux nœuds de l'arbre en parcourant en largeur.

            Cette fonction ajoute des timestamps aux nœuds de l'arbre en effectuant un parcours en largeur
            à partir du nœud racine.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX représentant l'arbre phylogénétique.
                root_id: L'identifiant du nœud racine à partir duquel effectuer le parcours en largeur.

            Returns:
                G (networkx.DiGraph): Le graphe NetworkX avec les timestamps attribués aux nœuds.
        """

        # Parcours en largeur de l'arbre
        bfs_tree = nx.bfs_predecessors(G,root_id)

        # Initialisation du compteur de timestamp
        ts = 5
        # Attribution des timestamps aux autres nœuds
        for node, parent in bfs_tree:
          if G.out_degree(node) != 0:
                # Attribution du timestamp unique au nœud
                G.nodes[node]['timestamp'] = ts
                # Incrémentation du compteur de timestamp
                ts += 5

        # Attribuer le max_timestamp à toutes les feuilles de G
        for node in G.nodes:
            if G.out_degree(node) == 0:  # Vérifier si le nœud est une feuille
                G.nodes[node]['timestamp'] = ts
            # Affichage timestamp de tous les noeuds
            print(f"Nœud {G.nodes[node]['number']}:{G.nodes[node]['timestamp']}")

        return G


    def display_timestamps(self, G):
        """
            Affiche les timestamps associés à chaque nœud du graphe G.

            Cette fonction parcourt tous les nœuds du graphe G et affiche leur numéro ainsi que le timestamp
            associé à chaque nœud.

            Args:
                G (networkx.Graph): Le graphe NetworkX contenant les nœuds avec leurs timestamps.

            Returns:
                None
        """
        print("------------------------Timestamp------------------------")
        for node in G.nodes():
            print(f"Nœud {G.nodes[node]['number']} : {G.nodes[node]['timestamp']}")



    def check_intersection(self, start1, end1, start2, end2):

        """
            Vérifie s'il y a une intersection entre deux intervalles donnés [start1, end1] et [start2, end2].

            Cette fonction prend en entrée les bornes inférieures (start1, start2) et supérieures (end1, end2) de deux intervalles
            [start1, end1] et [start2, end2] respectivement. Elle détermine s'il y a une intersection entre ces deux intervalles
            et renvoie True si une intersection est présente, sinon False.

            Args:
                start1 (int): La borne inférieure du premier intervalle.
                end1 (int): La borne supérieure du premier intervalle.
                start2 (int): La borne inférieure du deuxième intervalle.
                end2 (int): La borne supérieure du deuxième intervalle.

            Returns:
                bool: True si une intersection est présente, sinon False.
        """

        # Détermination de l'intervalle d'intersection
        start_1 = min(start1, end1)
        end_1 = max(start1, end1)
        start_2 = min(start2, end2)
        end_2 = max(start2, end2)

          # Si (G.nodes[pw]["timestamp"] < G.nodes[ay]["timestamp"]) and (G.nodes[w]["timestamp"] >= G.nodes[ay]["timestamp"]):
        return start_1 <= end_2 and start_2 <= end_1



    def calculate_mean_timestamp(self, G, node_1, node_2, node_3, node_4):
        """
        Calcule le mean_timestamp à partir des nœuds donnés.

        Cette fonction prend en entrée le graphe G et les identifiants de quatre nœuds node_1, node_2, node_3 et node_4.
        Elle calcule ensuite le mean_timestamp à partir des valeurs associées à ces nœuds dans le graphe G et le retourne.

        Args:
            G (networkx.Graph): Le graphe NetworkX contenant les nœuds avec leurs timestamps.
            node_1, node_2, node_3, node_4: Identifiants des nœuds pour lesquels on veut calculer le mean_timestamp.

        Returns:
            float: La valeur calculée du mean_timestamp.
        """
        start_timestamp = max(G.nodes[node_1]['timestamp'], G.nodes[node_3]['timestamp'])
        end_timestamp = min(G.nodes[node_2]['timestamp'], G.nodes[node_4]['timestamp'])
        mean_timestamp = (start_timestamp + end_timestamp) / 2

        return mean_timestamp



#----------------------------------------------------------

#----------------------------------------------------------



class TreeBuilder:
    def __init__(self):
        pass

    def build_graph_from_newick(self,newick_string):
        """
        Construit un graphe à partir d'une chaîne au format Newick.

        Cette fonction prend une chaîne au format Newick représentant un arbre phylogénétique,
        construit l'arbre correspondant à partir de cette chaîne, puis convertit l'arbre en un graphe
        utilisant la bibliothèque NetworkX.

        Args:
            newick_string (str): La chaîne au format Newick représentant l'arbre phylogénétique.

        Returns:
            G (networkx.DiGraph): Le graphe NetworkX représentant l'arbre phylogénétique.
            root_id: L'identifiant du nœud racine de l'arbre.

      """

        # Construction de l'arbre à partir du format Newick
        tree = Tree.parse_newick(newick_string)

        # Conversion de l'arbre en un graphe de NetworkX
        G, root_id = tree.to_nx()

        return G, root_id

    def initialize_graph(self, G,nbrCaractere): #TreeBuilder
        """
            Cette fonction initialise les attributs du graphe pour l'analyse phylogénétique en parcourant
            tous les nœuds et arêtes du graphe et en ajoutant les attributs nécessaires.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX représentant l'arbre phylogénétique.
                nbrCaractere (int): Le nombre de caractères pour l'analyse phylogénétique.

            Returns:
                G (networkx.DiGraph): Le graphe NetworkX initialisé avec les attributs nécessaires.
        """

        # Parcours de tous les nœuds de l'arbre
        for i, (node_id, node) in enumerate(G.nodes(data=True), start=1):
            # Ajout du numéro du nœud comme attribut
            node["number"] = i
            # Initialisation de l'attribut "timestamp" à 0 pour tous les nœuds
            node["timestamp"] = 0
            # Initialisation de l'attribut "type" à 'tree' pour tous les nœuds
            node["type"] = ''      #node["type"] = 'tree'

        # Initialisation de l'attribut "label" à None pour les feuilles uniquement
        feuilles = {node for node in G.nodes() if G.out_degree(node) == 0}
        for leaf in feuilles:
            G.nodes[leaf]["label"] = [None] * nbrCaractere

        # Parcours de toutes les arêtes de l'arbre
        for u, v in G.edges():
            G.edges[u, v]['type'] = 'support'
            G.edges[u, v]['color'] = 'black'

        return G

    def initialize_graph_without_leaf_label(self, G, nbrCaractere):
        """
            Cette fonction initialise les attributs du graphe pour l'analyse phylogénétique en parcourant
            tous les nœuds et arêtes du graphe et en ajoutant les attributs nécessaires, sauf l'attribut "label"
            qui ne sera pas initialisé pour les feuilles.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX représentant l'arbre phylogénétique.
                nbrCaractere (int): Le nombre de caractères pour l'analyse phylogénétique.

            Returns:
                G (networkx.DiGraph): Le graphe NetworkX initialisé avec les attributs nécessaires (sans "label" pour les feuilles).
        """

        # Parcours de tous les nœuds de l'arbre
        for i, (node_id, node) in enumerate(G.nodes(data=True), start=1):
            # Ajout du numéro du nœud comme attribut
            node["number"] = i
            # Initialisation de l'attribut "timestamp" à 0 pour tous les nœuds
            node["timestamp"] = 0
            # Initialisation de l'attribut "type" à 'tree' pour tous les nœuds
            node["type"] = 'tree'

        # Parcours de toutes les arêtes du graphe
        for u, v in G.edges():
            G.edges[u, v]['type'] = 'support'
            G.edges[u, v]['color'] = 'black'

        return G


    def generate_tree_networkX(self, nbrLeaf, nbrCaractere):
          """
              Orchestre un processus complet de génération, conversion et préparation d'un arbre phylogénétique
              pour l'analyse phylogénétique.

              Args:
                  nbrLeaf (int): Le nombre de feuilles de l'arbre phylogénétique.
                  nbrCaractere (int): Le nombre de caractères pour l'analyse phylogénétique.

              Returns:
                  tuple: Un tuple contenant deux éléments :
                      - Le graphe NetworkX représentant l'arbre phylogénétique préparé G (networkx.DiGraph).
                      - L'identifiant du nœud racine de l'arbre (root_id).
          """

          # Étape 1 : Génération de l'Arbre Phylogénétique
          result = species_tree_n(nbrLeaf, innovation=True)

          # Étape 2 : Conversion en Format Newick
          newick_string = to_newick(result)

          # Étape 3 : Construction du Graphe NetworkX
          tree_builder = TreeBuilder()
          G, root_id = tree_builder.build_graph_from_newick(newick_string)

          # Étape 4 : Initialisation des Attributs du Graphe
          G = tree_builder.initialize_graph(G, nbrCaractere)

          return G, root_id


    def drawGraph(self, G):
        """
            Dessine le graphe G avec ses nœuds et arêtes.

            Cette fonction utilise le layout kamada-kawai pour positionner les nœuds du graphe G
            et dessine les arêtes en utilisant les couleurs spécifiées dans l'attribut "color" des arêtes.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX à dessiner.

            Returns:
                None
        """

        # Dessin du graphe G
        pos = nx.kamada_kawai_layout(G)
        edge_colors = nx.get_edge_attributes(G, 'color').values()
        nx.draw(G, pos, with_labels=False, edge_color=edge_colors, arrowsize=20)
        node_labels = nx.get_node_attributes(G, "number")
        nx.draw_networkx_labels(G, pos, labels=node_labels)
        #plt.title("Graphe G")
        plt.show()


#----------------------------------------------------------

#----------------------------------------------------------


class LeafLabeler:
    def __init__(self):
        pass

    def set_leaf_attributes(self, G, nbrCaractere):
      """
          Définit les attributs des nœuds feuilles du graphe G avec les étiquettes existantes et initialise
          les attributs des autres nœuds à None.

          Args:
              G (networkx.DiGraph): Le graphe de l'arbre.
              nbrCaractere (int): Le nombre de caractères pour lesquels les nœuds feuilles seront étiquetés.

          Returns:
              dict: Dictionnaire contenant les attributs des nœuds, avec les nœuds feuilles étiquetés et les autres nœuds
              ayant leurs attributs initialisés à None.
      """

      attributes = {}
      # Extraction des étiquettes des feuilles
      leaf_labels = {node: G.nodes[node]["label"] for node in G.nodes() if G.out_degree(node) == 0}

      # Parcours de tous les nœuds du graphe G
      for node in G.nodes():
          if G.out_degree(node) == 0:
              # Si le nœud est une feuille, utilisez sa valeur de leaf_labels
              label = leaf_labels[node]
              attributes[G.nodes[node]["number"]] = {'Characters': label, 'spece': G.nodes[node]["number"]}
          else:
              # Si le nœud n'est pas une feuille, initialisez les valeurs à None
              attributes[G.nodes[node]["number"]] = {'Characters': [None]* (nbrCaractere), 'spece': None}

      return attributes


    def labeling_internal_nodes(self, G, attributes):
      """
          Effectue l'étiquetage des nœuds internes du graphe G en utilisant l'algorithme de parcours en profondeur.

          Cette fonction parcourt les nœuds du graphe G en utilisant le parcours en profondeur (DFS) post-ordre.
          Elle étiquette les nœuds internes du graphe en utilisant les étiquettes de leurs enfants directs.

          Args:
              G (networkx.DiGraph): Le graphe NetworkX contenant les nœuds et les arêtes.
              attributes (dict): Un dictionnaire contenant les attributs des nœuds, notamment les étiquettes à mettre à jour.

          Returns:
              dict: Le dictionnaire mis à jour contenant les attributs des nœuds après l'étiquetage des nœuds internes.
      """

      for node in nx.dfs_postorder_nodes(G):
          # Vérifier si le nœud n'a pas d'étiquetage
          if None in attributes[G.nodes[node]["number"]]['Characters']:
              # Récupération des enfants directs du nœud
              children = list(G.successors(node))

              # Cas où la racine n'a qu'un seul enfant
              if len(children) == 1 and G.in_degree(node) == 0:
                  child_label = attributes[G.nodes[children[0]]["number"]]['Characters']
                  attributes[G.nodes[node]["number"]]['Characters'] = child_label

              # Cas général avec deux enfants ou plus
              elif len(children) >= 2:
                  # Intersection des étiquettes des enfants pour chaque caractère
                  for c in range(len(attributes[G.nodes[node]["number"]]['Characters'])):
                      child_labels = [attributes[G.nodes[child]["number"]]['Characters'][c] for child in children]
                      attributes[G.nodes[node]["number"]]['Characters'][c] = reduce(lambda x, y: x & y, child_labels) # Équivalent à parent_label = child_labels[0] & child_labels[1] & ... & child_labels[n-1]

      return attributes


    def find_first_appearances(self, G, attributes, c):
      """
          Trouve et retourne l'ensemble des nœuds qui apparaissent en premier avec le caractère c dans le graphe G.

          Cette fonction parcourt les nœuds du graphe G et vérifie si un nœud possède le caractère c. Elle examine ensuite
          si le parent direct du nœud ne contient pas le caractère c. Si c'est le cas, le nœud est ajouté à l'ensemble Ac.

          Args:
              G (networkx.DiGraph): Le graphe NetworkX contenant les nœuds et les arêtes.
              attributes (dict): Un dictionnaire contenant les attributs des nœuds, notamment les étiquettes de caractères.
              c (int): Le caractère dont on cherche les premières apparitions.

          Returns:
              Ac (set()): L'ensemble des identifiants des nœuds qui apparaissent en premier avec le caractère c dans le graphe G.
      """

      Ac = set()
      for node in G.nodes:
          if G.nodes[node]["number"]!=1: #La racine

            # Si le nœud possède le caractère c
            if attributes[G.nodes[node]["number"]]['Characters'][c]== 1:

              # Déterminer le parent direct de notre nœud (NB: en travaille sur l'arbre G)
              parent = list(G.predecessors(node))[0]
              #  Vérifier si le parent du nœud ne contient pas le caractère
              if attributes[G.nodes[parent]["number"]]['Characters'][c] == 0:
                  # Si le nœud est une feuille, ajouter le nœud à l'ensemble Ac sans vérification des descendants
                  if G.out_degree(node) == 0:
                      Ac.add(node)
                  else:
                      # Si le nœud est un nœud interne, vérifier si tous les descendants directs possèdent le caractère c
                      descendants = list(G.successors(node)) # Tous les descendants directs (les enfants)
                      if all(attributes[G.nodes[descendant]["number"]]['Characters'][c] == 1 for descendant in descendants):
                          Ac.add(node)

      return Ac


    def display_leaf_labels(self, G):
      """
      Affiche les étiquettes des feuilles du graphe G.

      Args:
          G (networkx.Graph): Le graphe NetworkX.

      Returns:
          None
      """
      print("------------------------Leaf Labels------------------------")
      for node in G.nodes():
          if G.out_degree(node) == 0:
              label = G.nodes[node]["label"]
              print(f"Feuille {G.nodes[node]['number']}: {label}")


    def label_leaves_PTN(self, G, nbrCaractere):
      """
          Étiquetage des feuilles du réseau G pour qu'il avoir une solution PTN.

          Cette fonction attribue une étiquette (0 ou 1) à chaque feuille du graphe G
          pour chaque caractère spécifié dans la liste C. Le processus se répète pour chaque caractère
          dans C en effectuant les étapes suivantes :
          1. Choix aléatoire d'un nœud dans le graphe.
          2. Recherche des feuilles accessibles à partir du nœud choisi.
          3. Étiquetage des feuilles trouvées avec la valeur 1.
          4. Étiquetage des autres feuilles avec la valeur 0.

          Args:
              G (networkx.DiGraph): Le graphe NetworkX.
              nbrCaractere (int): Le nombre de caractères pour lesquels les nœuds feuilles seront étiquetés.

          Returns:
              G(networkx.DiGraph): Le graphe G avec les feuilles étiquetées pour chaque caractère de C.
      """

      for _ in range(nbrCaractere):
          # Choix aléatoire d'un nœud dans le graphe
          random_node = random.choice(list(G.nodes()))
          print("Nœud aléatoire du graphe: ",G.nodes[random_node]["number"])
          # Recherche des feuilles accessibles à partir du nœud choisi
          reachable_leaves = {node for node in nx.descendants(G, random_node) if G.out_degree(node) == 0}
          if G.out_degree(random_node) == 0:  # Ajouter le nœud random_node à l'ensemble reachable_leaves s'il a un degré de sortie égale à 0.
            reachable_leaves.add(random_node)

          # Étiquetage des feuilles trouvées avec la valeur 1
          for leaf in reachable_leaves:
              if "label" in G.nodes[leaf]:
                      if isinstance(G.nodes[leaf]["label"], list):
                          G.nodes[leaf]["label"].append(1)
                      else:
                          G.nodes[leaf]["label"] = [1]
              else:
                      G.nodes[leaf]["label"] = [1]


          # Étiquetage des autres feuilles avec la valeur 0
          for node in G.nodes():
              if G.out_degree(node) == 0 and node not in reachable_leaves:
                  if "label" in G.nodes[node]:
                          if isinstance(G.nodes[node]["label"], list):
                              G.nodes[node]["label"].append(0)
                          else:
                              G.nodes[node]["label"] = [0]
                  else:
                          G.nodes[node]["label"] = [0]

      return G



    def label_tree_leaves(self, G, nbrCaractere):
      """
          Étiquette les feuilles de l'arbre avec des valeurs aléatoires pour chaque caractère. (G peut être PTN ou Non-PTN)

          Args:
              G (networkx.DiGraph): Le graphe de l'arbre.
              nbrCaractere (int): Le nombre de caractères pour lesquels les feuilles seront étiquetées.

          Returns:
              G (networkx.DiGraph): Le graphe de l'arbre mis à jour avec les étiquettes ajoutées aux feuilles.
      """
      #feuilles = {node for node in G.nodes() if G.out_degree(node) == 0}
      for leaf in G.nodes():
          if G.out_degree(leaf) == 0:
              G.nodes[leaf]["label"] = [random.choice([0, 1]) for _ in range(nbrCaractere)]

      return G



    def label_leaves_tree(self, G, nbrCaractere):
      """
          Étiquette aléatoirement les feuilles de l'arbre pour établir une configuration compatible avec un réseau Non-PTN.

          Cette fonction effectue l'étiquetage aléatoire des feuilles d'un arbre phylogénétique afin de créer une configuration qui peut être associée à un réseau Non-PTN.
          Le processus garantit que les nœuds choisis répondent à certaines conditions pour préserver les propriétés du réseau Non-PTN.

          Args:
              G (networkx.DiGraph): Le graphe NetworkX représentant l'arbre phylogénétique.
              nbrCaractere (int): Le nombre de caractères pour l'analyse phylogénétique.

          Returns:
              tuple: Un tuple contenant trois éléments :
                  - Le graphe NetworkX mis à jour avec les feuilles étiquetées (G).
                  - L'identifiant du premier nœud choisi (node1).
                  - L'identifiant du deuxième nœud choisi (node2).
      """

      feuilles = {node for node in G.nodes() if G.out_degree(node) == 0}


      # Choix aléatoire d'un caractère
      caractere_choisi = random.randint(1, nbrCaractere)
      print("numero du caractere choisi: ",caractere_choisi )

       # Variable pour indiquer si les nœuds ont été trouvés
      found = False

      while not found:
              # Sélection aléatoire de deux nœuds du G non encore utilisés
              random_nodes = random.sample(list(G.nodes()), 2)
              node1 = random_nodes[0]
              node2 = random_nodes[1]


              # Vérification si les deux nœuds ne sont pas mutuellement descendants et n'ont pas de prédécesseurs en commun (pour les prédécesseurs directs)
              descendants_1 = set(nx.descendants(G, node1))
              descendants_2 = set(nx.descendants(G, node2))
              common_predecessors = set(G.predecessors(node1)) & set(G.predecessors(node2))

              reachable_leaves_node1=set()
              reachable_leaves_node2=set()
              reachable_leaves_union=set()
              # Recherche des feuilles accessibles à partir des deux nœuds choisis
              reachable_leaves_node1 = {node for node in nx.descendants(G, node1) if G.out_degree(node) == 0}
              if G.out_degree(node1) == 0:
                reachable_leaves_node1.add(node1)
              reachable_leaves_node2 = {node for node in nx.descendants(G, node2) if G.out_degree(node) == 0}
              if G.out_degree(node2) == 0:
                reachable_leaves_node2.add(node2)

              # Union des ensembles de feuilles accessibles
              reachable_leaves_union = (reachable_leaves_node1 | reachable_leaves_node2)

              # Verifier si Les nœuds satisfaisant les conditions
              if (node1 not in feuilles and node2 not in feuilles) and (node1 not in descendants_2 and node2 not in descendants_1) and (len(common_predecessors) == 0) and not feuilles <= reachable_leaves_union  :
                  found = True

      # Étiquetage des feuilles
      for leaf in G.nodes():
        if G.out_degree(leaf) == 0:
            # Si feuille est dans reachable_leaves_union: Étiqueter la feuille avec 1 pour le ’caractere_choisi’ et attribuer aléatoirement des valeurs de 0 ou 1 au reste de l’étiquetage.
            if leaf in reachable_leaves_union:
                G.nodes[leaf]["label"] = [random.choice([0, 1]) for _ in range(nbrCaractere)]
                G.nodes[leaf]["label"][caractere_choisi - 1] = 1
            # Sinon Étiqueter la feuille avec 0 pour le ’caractere_choisi’ et attribuer aléatoirement des valeurs de 0 ou 1 au reste de l’étiquetage.
            else:
                G.nodes[leaf]["label"] = [random.choice([0, 1]) for _ in range(nbrCaractere)]
                G.nodes[leaf]["label"][caractere_choisi - 1] = 0


      return G, node1, node2



    def possible_etiquetage(self, G, Gc, attributes, C, c):
        """
            Recherche les racines de G^c, les feuilles de G^c, puis vérifie si l'ensemble des feuilles est inclus
            dans les descendants pour chaque racine. Si cette condition est vérifiée, la racine est identifiée, et
            les étiquettes l des nœuds dans Rv sont mises à jour en ajoutant le caractère c. Sinon, les étiquettes
            des nœuds restants sont mises à jour en ajoutant la valeur 0 pour le caractère c.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX représentant l'arbre phylogénétique complet.
                Gc (networkx.DiGraph): Le sous-graphe de G correspondant à G^c.
                attributes (dict): Le dictionnaire des attributs des nœuds de l'arbre phylogénétique.
                C (list): La liste des caractères.
                c (int): L'index du caractère c.

            Returns:
                l (dict): Le dictionnaire mis à jour des attributs des nœuds après l'étiquetage. s'il est possible.
        """

        # Trouver les racines de G^c en utilisant l'in-degree
        X = [node for node, in_degree in Gc.in_degree() if in_degree == 0]
        print(f"Racines de G^{c}:", X)

        # Recherche des feuilles de G^c en utilisant le degré des nœuds
        leaves = [node for node in Gc.nodes if Gc.out_degree(node) == 0]
        print(f"Feuilles de G^{c}:", leaves)

        # Parcours des racines de G^c
        for node in X:
            Rv = set(nx.dag.descendants(Gc, node)) # Récupération des descendants de node dans G^c
            print(f"Rv(G^{c}): {Rv}")
            Rv.add(node) # Ajout de node à l'ensemble Rv

            # Vérification si toutes les feuilles (X) sont dans Rv
            if set(leaves).issubset(Rv):
                print(f"L(G^c)⊆ Rv(G^c), avec Racine v: {node}")

            # Mise à jour des attributs des nœuds de Gc avec le caractere c
                for n in Rv:
                    attributes[G.nodes[n]["number"]]['Characters'][len(C)] = 1
                else:
                  # Mise à jour des étiquettes des nœuds restants avec la valeur 0 pour le caractère c1
                    remaining_nodes = set(G.nodes) - set(Rv)
                    for n in remaining_nodes:
                        attributes[G.nodes[n]["number"]]['Characters'][len(C)] = 0

                l = attributes
                break
        # Si nous avons parcouru toutes les racines possibles dans X et n'avons trouvé aucune racine pour G^c.
        else:
            print("Aucune racine v trouvée OU L(G^c) /⊆ Rv(G^c)")
            l = None
            print("l'étiquetage n'est pas possible")

        return l

    def label_tree_leaves_species(self, species, nbrCaractere, dic):
        """
            Réalise l'étiquetage des feuilles d'un graphe à partir d'un dictionnaire.

            Cette fonction prend en entrée un graphe G, le nombre de caractères souhaité (nbrCaractere),
            ainsi qu'un dictionnaire nommé "dic" qui associe des étiquettes spécifiques à chaque feuille du graphe,
            en fonction de leur identifiant KEGG.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX représentant le réseau arborescent.
                nbrCaractere (int): Le nombre de caractères à utiliser pour l'étiquetage.
                dic (dict): Un dictionnaire associant des étiquettes aux identifiants KEGG des feuilles.

            Returns:
                G (networkx.DiGraph): Le graphe NetworkX mis à jour avec les étiquettes appliquées aux feuilles.

            Preconditions:
                Les feuilles du graphe G sont correctement définies.
                Le dictionnaire "dic" contient des correspondances pour les identifiants KEGG des feuilles.
        """

        for leaf in species.nodes():
            if species.out_degree(leaf) == 0:
                kegg_id = species.nodes[leaf]['kegg_id']
                label = dic.get(kegg_id, "")[:nbrCaractere]
                species.nodes[leaf]["label"] = label

        return species




#----------------------------------------------------------

#----------------------------------------------------------

class TreeBasedNetworks:
    def __init__(self):
        pass

    def compute_Fc(self, G, attributes, C):
          """
            Calcule l'ensemble Fc pour le graphe G avec l'étiquetage σ.

            L'ensemble Fc est défini comme l'ensemble des nœuds v appartenant à V(G) tels qu'il existe un nœud w appartenant à L(TG(v)),
            où c n'appartient pas à l'étiquetage σ(w). En d'autres termes, cela identifie les nœuds ayant au moins un descendant sans
            le caractère c dans leur étiquetage.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX.
                attributes (dict): Le dictionnaire des attributs des nœuds.
                C (list): La liste des caractères.

            Returns:
                Fc (set): L'ensemble des nœuds satisfaisant les conditions définies.
          """
          Fc = set()
          for node in nx.dfs_postorder_nodes(G):

            if G.out_degree(node) == 0: # Commencer par une feuille pour garantir le bon fonctionnement de dfs_postorder_nodes (NB: G c'est le réseau complet)
                # Si la feuille n'est pas dans Fc et ne possede pas le caractére c
                if node not in Fc and attributes[G.nodes[node]["number"]].get('Characters') and attributes[G.nodes[node]["number"]]['Characters'][len(C)] == 0:
                  # Ajouter la feuille à Fc
                  Fc.add(node)

                  # On débute la recherche à partir de cette feuille
                  current_node = node
                  parent = None
                  searching_parent = True

                  while searching_parent:
                      parents = set(G.predecessors(current_node))
                      # trouver le parent (qu'est unique) de current_node tel que l'arête (p, current_node) soit de type 'support'. Si une telle arête est trouvée, nous sortons de la boucle.
                      for p in parents:
                          if G.edges[p, current_node]['type'] == 'support':
                              parent = p
                              break
                      # Si on n'a pas trouvé de parent ou si le parent est déjà dans l'ensemble Fc, nous arrêtons de chercher plus haut dans l'arbre pour ce noeud .
                      if parent is None or parent in Fc:
                          searching_parent = False
                      # Sinon, on ajoute ce noeud, et on continue à chercher plus haut dans l'arbre
                      else:
                          Fc.add(parent)
                          current_node = parent
          return Fc


    def findLabeling(self, G, attributes, C):
        """
            Résout le problème de reconnaissance d'évolution de Phylogenetic Network (PTN).

            Le but est de trouver un étiquetage pour les nœuds internes du réseau arborescent G
            qui explique correctement l'évolution d'un ensemble donné d'espèces spécifié par l'ensemble
            de taxons S et les caractères C.

            Args:
                G (networkx.DiGraph): Le graphe NetworkX représentant le réseau arborescent G avec les nœuds et les arêtes.
                attributes (dict): Un dictionnaire contenant les attributs des nœuds du réseau G. Il doit contenir les caractères
                                  étiquetant les nœuds internes du réseau.
                C (list): Une liste de caractères C représentant l'ensemble des caractères disponibles
                        pour l'étiquetage des nœuds internes du réseau G.

            Returns:
                dict or None: Un dictionnaire contenant les attributs mis à jour des nœuds du réseau G.
                Les caractères des nœuds internes sont mis à jour en fonction de l'étiquetage trouvé pour
                expliquer l'évolution des espèces spécifiée par l'ensemble de taxons S. Si aucun étiquetage
                n'est possible, renvoie None.

            Preconditions:
                Les bords de support et de transfert du réseau G sont connus.
        """

        # Dessin du graphe G
        plt.figure()
        plt.title(f"Graphe G")
        tree_builder = TreeBuilder()
        tree_builder.drawGraph(G)

        l = 1

        # Vérification de la condition "l not NULL and C is not empty"
        while l is not None and len(C) > 0:
            # Sélection d'un caractère c dans C
            c = C.pop()

            # Calcule l'ensemble Fc pour le graphe G
            tree_networks= TreeBasedNetworks()
            Fc = tree_networks.compute_Fc(G, attributes, C)

            # Affichage des nœuds ayant au moins un descendant sans le caractère c
            print("----------------------------------------------------------------------------------------------------------")
            print(f"Nœuds ayant au moins un descendant sans le caractère {c} F{c}:", [G.nodes[node]['number'] for node in Fc] )


            # Création du graphe G^c pour le caractère c
            Gc = G.copy()
            Gc.remove_nodes_from(Fc)

            # Affichage du graphe Gc pour le caractère c
            print(f"Graphe G\u0302{c}:")
            print(f"les noeuds du G\u0302{c}: {[Gc.nodes[node]['number'] for node in Gc.nodes]}")
            print()

            plt.figure()
            plt.title(f"Graphe G\u0302{c}")
            tree_builder.drawGraph(Gc)

            # Si Gc est vide, on attribue la valeur 0 au caractère c pour tous les nœuds de G.
            if len(Gc) == 0:
              for n in G.nodes():
                 attributes[G.nodes[n]["number"]]['Characters'][len(C)] = 0
            else:
              # Vérification de la connectivité faible du graphe G^c
              is_connected = nx.is_weakly_connected(Gc)

              # Si le graphe G^c n'est pas faiblement connecté, l'étiquetage n'est pas possible pour ce caractère c.
              if not is_connected:
                  print(f"Le graphe G\u0302{c} n'est pas faiblement connecté.")
                  l = None
                  print("l'étiquetage n'est pas possible")
              # Si le graphe G^c est faiblement connecté, on essaie de trouver un étiquetage possible
              else:
                  leaflabeler=LeafLabeler()
                  l= leaflabeler.possible_etiquetage(G, Gc, attributes, C, c)

            print()
            print(l)
            print()

        return l



    def TransferAdditionGreedy(self, G, attributes):
        """
          Algorithme greedy pour ajouter les transferts aux nœuds de l'arbre en fonction des attributs des nœuds afin d'avoir un TBN PTN.

        Args:
            G (networkx.Graph): Le graphe de l'arbre.
            attributes (dict): Dictionnaire contenant les attributs des nœuds de l'arbre.

        Returns:
            tuple: Un tuple contenant deux éléments :
                - Le graphe de l'arbre mis à jour avec les transferts ajoutés (G).
                - Le dictionnaire des attributs des nœuds mis à jour avec les transferts ajoutés (attributes).
        """

        # Étape 1 : Labeling des nœuds internes de l'arbre
        labeler = LeafLabeler()
        attributes = labeler.labeling_internal_nodes(G, attributes)

        print(attributes)

        # Nombre de caractères à analyser
        nbrCaractere = len(attributes[1]['Characters'])

        # Parcours pour chaque caractère
        for c in range(nbrCaractere):
            print("Caractère:", c)

            # Étape 6 : Déterminer "The first appearance list" => Ac
            Ac = labeler.find_first_appearances(G, attributes, c)
            print("Ac: ",{G.nodes[node]["number"] for node in Ac})

            # Étape 8 : Triez l'ensemble Ac dans l'ordre croissant selon le "timestamp"
            Xc = list(sorted(Ac, key=lambda node: G.nodes[node]["timestamp"])) #NB: un ensemble, qui par définition ne conserve pas l'ordre des éléments donc on va utiliser une liste pour le tri
            print("Xc: ",[G.nodes[node]["number"] for node in Xc])


            # Étape 9 : Parcours des nœuds de Xc
            for i in range(len(Xc)-1):

              # ai  F.a  et son parent c’est ai’
              ai = Xc[i]
              # Trouver le parent direct de ai (ai') qui est un nœud de support
              for predecessor in G.predecessors(ai):
                if G.edges[predecessor, ai]['type'] == 'support':
                    pai = predecessor
                    break
              # ay (ai+1)  F.a  et son parent c’est pay (ai+1’)
              ay = Xc[i+1]
              # Trouver le parent direct de ay (ai+1') qui est un nœud de support
              for predecessor in G.predecessors(ay):
                if G.edges[predecessor, ay]['type'] == 'support':
                    pay = predecessor
                    break


              # Extraction des BFS_tree sans transfert
              transfer_manager = TransferManager()
              bfs_nodes = transfer_manager.extract_bfs_tree_without_transfer(G, ai)
              print(f"bfs_nodes: {[G.nodes[node]['number'] for node in bfs_nodes]}")


              # Existe-t-il déjà un transfert entre w et exactement ai+1^ ?

              for w in bfs_nodes:

                for predecessor in G.predecessors(w):
                  if G.edges[predecessor, w]['type'] == 'support':
                      pw = predecessor
                      break

                if G.has_edge(pw, pay) and G.edges[pw, pay]['type'] == 'transfer':

                      print(f"Il existe déjà un transfert entre {G.nodes[pw]['number']} et {G.nodes[ay]['number']}")

                      # Ajouter 1 pour le caractere c (pour les noeuds dans le chemin entre w^ jusqu'à ay)
                      c_list = [1 if i == c else 0 for i in range(len(attributes[G.nodes[pw]["number"]]['Characters']))]
                      shortest_path = nx.shortest_path(G, pw, ay)
                      for node in shortest_path:
                        attributes[G.nodes[node]["number"]]['Characters'] = [a | b for a, b in zip(attributes[G.nodes[node]["number"]]['Characters'], c_list)]

                      break

              ## 2) Sinon, Aucun transfert n'a été trouvé pour tous les nœuds de bfs_node, on va ajouter un transfert
              else: #12
                  print("Aucun transfert n'a été trouvé pour les nœuds de bfs_nodes")

                  for w in bfs_nodes:
                    for predecessor in G.predecessors(w):
                      if G.edges[predecessor, w]['type'] == 'support':
                          pw = predecessor
                          break

                    # Déterminer la début et la fin de chaque intervalle
                    start_1 = G.nodes[pw]['timestamp']
                    end_1 = G.nodes[w]['timestamp']
                    start_2 = G.nodes[pay]['timestamp']
                    end_2 = G.nodes[ay]['timestamp']

                    # Vérifier s'il y a une intersection entre les deux intervalles
                    timestamp_manager = TimestampManager()
                    if timestamp_manager.check_intersection(start_1, end_1, start_2, end_2):
                      # Il y a une intersection entre les intervalles, on va ajouter notre transfert
                      G=transfer_manager.add_transfer_to_tree(G, pw, w, pay, ay, c, attributes) #13

                      break # Si un seul transfert a été appliqué, on sort de la boucle For


        return G, attributes




